# A Stack Overflow Story

## Lab Assignment Two: Exploring Text Data 

### Justin Ledford, Luke Wood, Traian Pop 

In [21]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=2)
#BASEURL = "https://api.stackexchange.com/2.2/questions?site=stackoverflow&key=ckDNIw23urCJMtURIGBSIQ(("
#/2.2/questions?fromdate=1451606400&todate=1483228800&order=desc&sort=activity&site=stackoverflow
BASEURL="https://api.stackexchange.com/2.2/questions"
params = {
  "site" : "stackoverflow",
  "key" : "ckDNIw23urCJMtURIGBSIQ((",
  "fromdate" : "1451606400",
  "todate" : "1483228800",
  "order" : "desc",
  "sort" : "activity",
  "filter" : "withbody" #shows question and answer text
}

r = requests.get(BASEURL, params=params)

questions = r.json()

pp.pprint(questions)


{ 'has_more': True,
  'items': [ { 'answer_count': 1,
               'body': '<p>Hello i want to grab image using pyscreenshot '
                       "libary and then stream it using flask my code don't "
                       'works for some reason\n'
                       'my code:</p>\n'
                       '\n'
                       '<h1>camera.py</h1>\n'
                       '\n'
                       '<pre><code>import cv2\n'
                       'import pyscreenshot as ImageGrab\n'
                       '\n'
                       'class VideoCamera(object):\n'
                       'def __init__(self):\n'
                       '    # Using OpenCV to capture from device 0. If you '
                       'have trouble capturing\n'
                       '    # from a webcam, comment the line below out and '
                       'use a video file\n'
                       '    # instead.\n'
                       '    self.video = cv2.VideoCapture(0)\n'
        

In [23]:
question_list = [x['body'] for x in questions['items']]
question_list


["<p>Hello i want to grab image using pyscreenshot libary and then stream it using flask my code don't works for some reason\nmy code:</p>\n\n<h1>camera.py</h1>\n\n<pre><code>import cv2\nimport pyscreenshot as ImageGrab\n\nclass VideoCamera(object):\ndef __init__(self):\n    # Using OpenCV to capture from device 0. If you have trouble capturing\n    # from a webcam, comment the line below out and use a video file\n    # instead.\n    self.video = cv2.VideoCapture(0)\n    # If you decide to use video.mp4, you must have this file in the folder\n    # as the main.py.\n    # self.video = cv2.VideoCapture('video.mp4')\n\ndef __del__(self):\n    self.video.release()\n\ndef get_frame(self):\n    success, image = self.video.read()\n    # We are using Motion JPEG, but OpenCV defaults to capture raw images,\n    # so we must encode it into JPEG in order to correctly display the\n    # video stream.\n    ret, jpeg = cv2.imencode('.jpg', image)\n    return jpeg.tobytes()\n\ndef get_screen_frame(se